In [25]:
import pandas as pd
import pickle
import numpy as np
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import random
from Class_Balancing import *

In [30]:
#  Location of file to convert
llama_scope = "simple"
df = pd.read_csv(rf"C:\Users\Maciek\Downloads\llama_generated_{llama_scope}_amazon_review.csv")
df = pd.concat([df,pd.read_csv(rf"C:\Users\Maciek\Downloads\llama_generated_{llama_scope}_amazon_review (1).csv")])
df = pd.concat([df,pd.read_csv(rf"C:\Users\Maciek\Downloads\llama_generated_{llama_scope}_amazon_review (2).csv")])
generated_text = df.text
generated_text

0       This Huawei tablet has exceeded my expectatio...
1       The Plantronics tablet is an excellent device...
2       Asus has done it again! This tablet is a beas...
3       I recently purchased the E Fun tablet and I a...
4       I recently purchased the Apple iPad and I am ...
                             ...                        
995     This Samsung tablet is a game changer! The di...
996     This Huawei tablet is a game changer! The dis...
997     I recently purchased the Google Pixel C table...
998     The Huawei tablet is an absolute beast! The b...
999     This Dell tablet has exceeded my expectations...
Name: text, Length: 7000, dtype: object

### Removal of noise


In [31]:
def remove_noise(x):
    if len(x.strip())==0:
        return None
    """method used for removal of instances that contain mostly repeating tokens ex. "MS MS MS MS ..."

    Args:
        x (str): text to check

    Returns:
        str: input text if tokens are not repeating else None
    """
    k, v = np.unique(x.split(), return_counts=True)
    if np.max(v)>len(k):
        print(x)
        return None
    return x

generated_text = generated_text.apply(lambda x:remove_noise(x))
generated_text = list(generated_text[~generated_text.isna()])
len(generated_text)

7000

In [28]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, evidences=None):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.evidences=evidences

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
#             token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])   

In [29]:
i = 0
random_seed = 123

np.random.seed(random_seed)
random.seed(random_seed)
for imbalance in [50, 20, 10]:
    for i in range(5):
        # Loading imbalanced datasets
        with open(rf"C:\Users\Maciek\Documents\Studia\Magisterka\Amazon\nlpaug\amazon_review_{imbalance}_imbalanced_{i}.obj", 'rb') as pickle_file:
            imbalanced_dataset = pickle.load(pickle_file)
        
        # Combining imbalanced data and LLama generated revies
        generation_count = len(imbalanced_dataset.targets) - 2*np.sum(imbalanced_dataset.targets)
        print(generation_count)
        balanced_inputs = imbalanced_dataset.inputs + random.sample(generated_text, generation_count)
        balanced_targets = imbalanced_dataset.targets + [1 for _ in range(generation_count)]

        # Shuffling inputs and targets after balancing
        np.random.seed(random_seed)
        np.random.shuffle(balanced_targets)
        np.random.seed(random_seed)
        np.random.shuffle(balanced_inputs)

        # Creating the dataset
        dataset = TrainerDataset(balanced_inputs, balanced_targets, imbalanced_dataset.tokenizer)
        
        # Saving created dataset to file
        filehandler = open(rf'C:\Users\Maciek\Documents\Studia\Magisterka\Amazon\nlpaug\erqaser_movie_{imbalance}_LLama_{llama_scope}_{i}.obj',"wb")
        pickle.dump(dataset,filehandler)
        filehandler.close()
        
        _, counts = np.unique(dataset.targets, return_counts=True)
        ratio = counts[0]/np.sum(counts)
        print(f"Ratio: {ratio}")
        print("Some examples from positive class")
        print(np.sort((np.array(dataset.inputs)[np.array(dataset.targets)==1]))[-3:])
        print("========================================================================================================================================================")
        print()
        print()

636
Ratio: 0.5
Some examples from positive class
["xoom rocks! Lovin the flash support and expandable memory, the two main things I wanted in a tablet. Xoom can do more than the ipad:-) but that's just me."
 "xoom xoom First of all I was looking for a replacement for my laptop cause I am a 'tech junkie' and didn't feel too good walking around with a bulky laptop and then I saw the advertisment of the the xoom. I thought it looked like sum thing from a sci fi film. So I did my research and liked everything in read about it. Finally I got it about two weeks after I ordered it from amazon and was breath-takenly surprised at how smooth it was...fast,quick respnsive and even made my shifts at my job so much more enjoyable. With rgards to hardware,the xoom has that covered... software could do a whole lot better thoMOTOROLA XOOM Android Tablet (Wi-Fi)ugh. I was kinda pissed at the fact that I couldn't use the memory card slot until an update was out, or I couldn't connect a USB storage devic